In [1]:
import refinitiv.data as rd
import eikon as ek
import pandas as pd
from datetime import datetime
import datetime as dt
import dateutil.relativedelta

In [2]:
# set session for Refinitiv 
ek.set_app_key('93b5caec34434c3497e824215fb7fe32e1f211f7')
rd.open_session()

<refinitiv.data.session.Definition object at 0xe694c3ea40 {name='workspace'}>

# 1. Daily Return for S&P

In [3]:
# get company list
ic, err = ek.get_data('.SPX', ['TR.IndexConstituentRIC'])
lj, err = ek.get_data('.SPX', 
                      ['TR.IndexJLConstituentChangeDate',
                       'TR.IndexJLConstituentRIC.change',
                       'TR.IndexJLConstituentRIC'],
                     {'SDate':'0D', 'EDate':'-31Y', 'IC':'B'}) ##start by 1957
lj['Date']=pd.to_datetime(lj['Date']).dt.date
lj.sort_values(['Date','Change'],ascending=False,inplace=True)
dates = [dt.date(1995,1,31)]
i = 0
while (dates[0] + dateutil.relativedelta.relativedelta(months=+i+1)) < dt.date.today():
    dates.append(dates[0] + dateutil.relativedelta.relativedelta(months=+i+1))
    i = i + 1
dates.append(dt.date.today())
df = pd.DataFrame(index=dates, columns=['Index Constituents'])

In [4]:
filtered_date = []
start_date = datetime(1995, 1, 1).date()
end_date = datetime(2020, 12, 31).date()
for date_obj in dates:
    if start_date <= date_obj <= end_date:
        filtered_date.append(date_obj)
date_strings = [date.strftime('%Y%m%d') for date in filtered_date]
date_strings

['19950131',
 '19950228',
 '19950331',
 '19950430',
 '19950531',
 '19950630',
 '19950731',
 '19950831',
 '19950930',
 '19951031',
 '19951130',
 '19951231',
 '19960131',
 '19960229',
 '19960331',
 '19960430',
 '19960531',
 '19960630',
 '19960731',
 '19960831',
 '19960930',
 '19961031',
 '19961130',
 '19961231',
 '19970131',
 '19970228',
 '19970331',
 '19970430',
 '19970531',
 '19970630',
 '19970731',
 '19970831',
 '19970930',
 '19971031',
 '19971130',
 '19971231',
 '19980131',
 '19980228',
 '19980331',
 '19980430',
 '19980531',
 '19980630',
 '19980731',
 '19980831',
 '19980930',
 '19981031',
 '19981130',
 '19981231',
 '19990131',
 '19990228',
 '19990331',
 '19990430',
 '19990531',
 '19990630',
 '19990731',
 '19990831',
 '19990930',
 '19991031',
 '19991130',
 '19991231',
 '20000131',
 '20000229',
 '20000331',
 '20000430',
 '20000531',
 '20000630',
 '20000731',
 '20000831',
 '20000930',
 '20001031',
 '20001130',
 '20001231',
 '20010131',
 '20010228',
 '20010331',
 '20010430',
 '20010531',

In [5]:
# download company in S&P for each end of month

i=0
result_df = pd.DataFrame()
for date_str in date_strings:
    isSuccess = False
    while(not isSuccess):
        try:
            df, e = ek.get_data(f'0#.SPX({date_str})', ['TR.RIC'])
            df['Date']=date_str
            isSuccess = True
        except ek.EikonError as err:
            continue
        df = df.drop(['Instrument'],axis=1)
        result_df = pd.concat([result_df, df], ignore_index=True)
        print('Complete:'+ str(date_str))
print(result_df)

Complete:19950131
Complete:19950228
Complete:19950331
Complete:19950430
Complete:19950531
Complete:19950630
Complete:19950731
Complete:19950831
Complete:19950930
Complete:19951031
Complete:19951130
Complete:19951231
Complete:19960131
Complete:19960229
Complete:19960331
Complete:19960430
Complete:19960531
Complete:19960630
Complete:19960731
Complete:19960831
Complete:19960930
Complete:19961031
Complete:19961130
Complete:19961231
Complete:19970131
Complete:19970228
Complete:19970331
Complete:19970430
Complete:19970531
Complete:19970630
Complete:19970731
Complete:19970831
Complete:19970930
Complete:19971031
Complete:19971130
Complete:19971231
Complete:19980131
Complete:19980228
Complete:19980331
Complete:19980430
Complete:19980531
Complete:19980630
Complete:19980731
Complete:19980831
Complete:19980930
Complete:19981031
Complete:19981130
Complete:19981231
Complete:19990131
Complete:19990228
Complete:19990331
Complete:19990430
Complete:19990531
Complete:19990630
Complete:19990731
Complete:1

In [6]:
result_df.head()

,RIC,Date
0,MMM.N,19950131
1,CCB.N^B96,19950131
2,TNB.N^E12,19950131
3,ABT.N,19950131
4,ADP.OQ,19950131


In [7]:
unique_values = result_df['RIC'].unique()
unique_values

<StringArray>
[     'MMM.N',  'CCB.N^B96',  'TNB.N^E12',      'ABT.N',     'ADP.OQ',
     'AEP.OQ',  'CSR.N^F00',  'ANV.N^D99',  'AET.N^K18', 'USHC.O^G96',
 ...
      'DPZ.N',      'WST.N',      'BIO.N',      'TDY.N',      'TYL.N',
     'CTLT.N',    'ETSY.OQ',    'POOL.OQ',      'VNT.N',    'TSLA.OQ']
Length: 1153, dtype: string

In [8]:
result_df.to_csv('month_end.csv')

In [5]:
ic_list = ic['Constituent RIC'].tolist()
for i in range(len(dates)):
    print(str(dates[len(dates)-i-1]))
    df.at[dates[len(dates)-i-1],'Index Constituents'] = ic_list[:]
    for j in lj.index:
        if lj['Date'].loc[j] <= dates[len(dates)-i-1]:
            if lj['Date'].loc[j] > dates[len(dates)-i-2]:
                if lj['Change'].loc[j] == 'Joiner':
                    print('Removing ' + lj['Constituent RIC'].loc[j])
                    ic_list.remove(lj['Constituent RIC'].loc[j])
                elif lj['Change'].loc[j] == 'Leaver':
                    print('Adding ' + lj['Constituent RIC'].loc[j])
                    ic_list.append(lj['Constituent RIC'].loc[j])
            else:
                break

2023-09-07
2023-08-30
Adding AAP.N
Removing KVUE.N
2023-07-30
Adding FTRE.OQ
Adding PHIN.N
Removing PHIN.N
Removing FTRE.OQ
2023-06-30
Adding DISH.OQ
Removing PANW.OQ
2023-05-30
Adding FRC.N^E23
Removing AXON.OQ
2023-04-30
2023-03-30
Adding LUMN.N
Removing FICO.N
Adding SIVB.OQ^C23
Adding SBNY.OQ^C23
Removing BG.N
Removing PODD.OQ
2023-02-28
2023-01-30
Adding VNO.N
Removing GEHC.OQ
2022-12-30
Adding ABMD.OQ^L22
Removing STLD.OQ
Adding MBC.N
Adding FBIN.N
Removing FSLR.OQ
Removing MBC.N
2022-11-30
Adding TWTR.N^J22
Removing ACGL.OQ
2022-10-30
Adding NLSN.N^J22
Removing TRGP.N
Adding CTXS.OQ^J22
Adding DRE.N^J22
Removing PCG.N
Removing EQT.N
2022-09-30
Adding PVH.N
Adding PENN.OQ
Removing CSGP.OQ
Removing INVH.N
2022-08-30
2022-07-30
2022-06-30
Adding UAA.N
Adding UA.N
Adding IPGP.OQ
Removing KDP.OQ
Removing ON.OQ
Adding CERN.OQ^F22
Removing VICI.N
2022-05-30
2022-04-30
Adding DISCK.OQ^D22
Adding EMBC.OQ
Adding PBCT.OQ^D22
Removing CPT.N
Removing EMBC.OQ
2022-03-30
Adding INFO.N^B22
Addi

2013-02-28
Adding BIG.N
Removing PVH.N
2013-01-30
Adding FHI.N
Removing ABBV.N
2012-12-30
Adding TIE.N^A13
Removing APTV.N
Adding RRD.N^B22
Removing GRMN.N
Adding CBE.N^L12
Removing DG.N
2012-11-30
2012-10-30
Adding SUN.N^J12
Removing PETM.OQ^C15
Adding ANR.N^G15
Removing KRFT.OQ^G15
Adding LXK.N^K16
Adding ATGE.N
Removing PNR.N
Removing ADT.N^E16
2012-09-30
Adding SHLD.OQ^J18
Removing LYB.N
2012-08-30
Adding GR.N^G12
Removing VAL.N^H20
2012-07-30
Adding PGN.N^G12
Removing STX.OQ
2012-06-30
Adding HSH.N^H14
Removing MNST.OQ
Adding NVLS.OQ^F12
Removing LRCX.OQ
2012-05-30
Adding EP.N^E12
Removing KMI.N
Removing ALXN.OQ^G21
Adding MMI.N^E12
Removing PSX.N
Adding SVU.N^J18
2012-04-30
Removing FOSL.OQ
Adding MHS.N^D12
2012-03-30
Removing CCI.N
Adding CEG.N^C12
2012-02-29
2012-01-30
Adding CPWR.OQ^L14
Removing WPX.N^A21
2011-12-30
Adding TLAB.OQ^L13
Removing TRIP.OQ
Adding AKS.N^C20
Adding MWW.N^K16
Adding SUNE.N^D16
Removing PRGO.N
Removing BWA.N
Removing DLTR.OQ
Removing GAS.N^G16
Adding G

Adding AWE.N^J04
Removing CIT.N^K09
2004-09-30
Adding CF.N^H04
Removing TPR.N
2004-08-30
Adding TNB.N^E12
Removing FSH.N^K06
2004-07-30
Adding UPC.N^G04
Adding ONE.N^G04
Removing GILD.OQ
Removing SOV.N^A09
2004-06-30
2004-05-30
Adding AM.N^H13
Removing HSP.N^I15
2004-04-30
Adding JHF.N^D04
Removing VLO.N
Adding PCS.N^D04
Removing VTRS.OQ
Adding TAPb.N^D04
Removing ACS.N^B10
Adding FBF.N^D04
Removing ETFC.OQ^J20
2004-03-30
Adding TUP.N
Removing CMX.N^C07
2004-02-29
Adding CE.N^B04
Removing MTB.N
2004-01-30
2003-12-30
2003-11-30
Adding PEG.N
Removing PEG.N
Adding BGEN.OQ^K03
Removing BIIB.OQ
2003-10-30
2003-09-30
Adding QTRN.OQ^I03
Removing ESRX.OQ^L18
2003-08-30
Adding MDR.N^A20
Adding MHS_w.N^H03
Removing MHS_w.N^H03
Removing MHS.N^D12
2003-07-30
Adding MIR.N^G03
Removing PLD.N^F11
2003-06-30
2003-05-30
2003-04-30
Adding PHA.N^D03
Removing FHI.N
Adding MWW.N^K16
Removing MWW.N^K16
Adding HI.N^C03
Removing GEN.OQ
2003-03-30
Adding HRC.N^C03
Removing MKC.N
Adding AMR.N^A12
Removing AIV.N

Adding ITT.N^B98
Removing MTL.N^I99
1998-01-30
Adding PAS.N^C10
Removing NTRS.OQ
Adding CBB.N^A98
Removing SUB.N^C01
Adding ECO.A^B03
Removing BIG.N
Adding BBI.N^A98
Removing LEH.N^I08
Adding UEP.N^L97
Removing AEE.N
1997-12-30
Adding FLM.N^D03
Removing OMC.N
Adding HFS.N^L97
Removing CINF.OQ
Adding BEV.N^L97
Removing TFC.N
1997-11-30
Adding SB.N^K97
Removing SNV.N
Adding OEC.N^K97
Removing FE.N
1997-10-30
Adding LLX.N^J97
Removing HBOC.O^A99
Adding SRR.N^H07
Removing YUM.N
1997-09-30
Adding AMH.A^J97
Removing KLAC.OQ
Adding TDM.N^I97
Removing CCU.N^G08
1997-08-30
Adding NYN.N^H97
Removing STT.N
Adding FJ.N^K00
Removing FJ.N^K00
Adding NAE.N^H97
Removing MIR.N^E00
Removing HBAN.OQ
Adding ENS.N^H97
Adding MD.N^H97
Removing PGR.N
Removing OI.N
1997-07-30
Adding INGR.OQ^L06
Adding SFS.N^H00
Removing APA.OQ
Removing APC.N^H19
Adding GIDL.O^J97
Removing SAI.N^L98
Adding GWF.N^G97
Removing WM.N^I08
1997-06-30
Adding PEL.N^F97
Removing EFX.N
Adding USH.N^F97
Removing CFC.N^G08
Adding PVN.N^F9

In [28]:
List_Length = df['Index Constituents'].apply(len)
List_Length

1995-01-30    484
1995-02-28    484
1995-03-30    484
1995-04-30    484
1995-05-30    483
             ... 
2023-05-30    503
2023-06-30    503
2023-07-30    503
2023-08-30    503
2023-09-07    503
Name: Index Constituents, Length: 345, dtype: int64

In [14]:
df.index = pd.to_datetime(df.index)

# Filter the DataFrame based on the date range to remove 2020 afterward data
start_date = datetime(1995, 1, 1)
end_date = datetime(2020, 12, 31)
filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]

unique_constituents = set()
for constituents_list in filtered_df["Index Constituents"]:
    unique_constituents.update(constituents_list)

# Convert the set back to a sorted list
unique_constituents_list = sorted(list(unique_constituents))
print(len(unique_constituents_list))

1152


In [15]:
i=0
result = pd.DataFrame()
for i in range(len(unique_constituents_list)): #company list from 19950101 to 20201231
    isSuccess = False
    while(not isSuccess):
        try:
            data_grid, err = ek.get_data(
            list(unique_constituents_list)[i],
            fields = ['TR.TotalReturn1D.date','TR.TotalReturn1D'],
            parameters={'SDate':'1995-01-01','EDate':'2020-12-31','Frq':'D','CH':'Fd'}
            )
            isSuccess = True
        except ek.EikonError as err:
            continue
        result = pd.concat([result,data_grid],axis=0).reset_index(drop=True)
        print('Complete:'+ str(i) + '. '+list(pd.unique(lj['Constituent RIC']))[i])
        i+=1

Complete:0. AAP.N
Complete:1. KVUE.N
Complete:2. FTRE.OQ
Complete:3. PHIN.N
Complete:4. DISH.OQ
Complete:5. PANW.OQ
Complete:6. FRC.N^E23
Complete:7. AXON.OQ
Complete:8. LUMN.N
Complete:9. FICO.N
Complete:10. SBNY.OQ^C23
Complete:11. SIVB.OQ^C23
Complete:12. BG.N
Complete:13. PODD.OQ
Complete:14. VNO.N
Complete:15. GEHC.OQ
Complete:16. ABMD.OQ^L22
Complete:17. STLD.OQ
Complete:18. MBC.N
Complete:19. FBIN.N
Complete:20. FSLR.OQ
Complete:21. TWTR.N^J22
Complete:22. ACGL.OQ
Complete:23. NLSN.N^J22
Complete:24. TRGP.N
Complete:25. CTXS.OQ^J22
Complete:26. DRE.N^J22
Complete:27. EQT.N
Complete:28. PCG.N
Complete:29. PENN.OQ
Complete:30. PVH.N
Complete:31. CSGP.OQ
Complete:32. INVH.N
Complete:33. UAA.N
Complete:34. UA.N
Complete:35. IPGP.OQ
Complete:36. ON.OQ
Complete:37. KDP.OQ
Complete:38. CERN.OQ^F22
Complete:39. VICI.N
Complete:40. DISCK.OQ^D22
Complete:41. PBCT.OQ^D22
Complete:42. EMBC.OQ
Complete:43. CPT.N
Complete:44. ZIMV.OQ
Complete:45. INFO.N^B22
Complete:46. MOH.N
Complete:47. XLN

2023-09-04 13:01:08,954 P[7752] [MainThread 7748] HTTP TimeoutException: .


Complete:314. CNX.N
Complete:315. AWK.N
Complete:316. PCL.N^B16
Complete:317. PCP.N^B16
Complete:318. BRCM.OQ^B16
Complete:319. FRT.N
Complete:320. CFG.N
Complete:321. CB.N^A16
Complete:322. EXR.N
Complete:323. FOSL.OQ
Complete:324. WTW.OQ
Complete:325. ALTR.OQ^L15
Complete:326. CHD.N


2023-09-04 13:02:27,178 P[7752] [MainThread 7748] Backend error. 400 Bad Request
2023-09-04 13:02:27,178 P[7752] [MainThread 7748] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:327. CMCSK.OQ^L15
Complete:328. SIAL.OQ^K15
Complete:329. ILMN.OQ
Complete:330. GNW.N
Complete:331. SYF.N
Complete:332. FCPT.N
Complete:333. HCBK.OQ^K15
Complete:334. HPE.N
Complete:335. JOY.N^D17
Complete:336. VRSK.OQ
Complete:337. NWS.OQ
Complete:338. HSP.N^I15
Complete:339. UAL.OQ
Complete:340. PLL.N^H15
Complete:341. ATVI.OQ
Complete:342. DTV.OQ^G15
Complete:343. NE.N^H20
Complete:344. PYPL.OQ
Complete:345. FDO.N^G15
Complete:346. KRFT.OQ^G15
Complete:347. KHC.OQ
Complete:348. ATI.N
Complete:349. MWV.N^G15
Complete:350. WRK.N
Complete:351. QEP.N^C21
Complete:352. TEG.N^F15
Complete:353. JBHT.OQ
Complete:354. LO.N^F15
Complete:355. QRVO.OQ
Complete:356. WIN.OQ^C19
Complete:357. O.N
Complete:358. AGN.N^C15
Complete:359. AVP.N^A20
Complete:360. DNR.N^G20
Complete:361. NBR.N
Complete:362. AAL.OQ
Complete:363. EQIX.OQ
Complete:364. CFN.N^C15
Complete:365. HSIC.OQ
Complete:366. PETM.OQ^C15
Complete:367. SWKS.OQ
Complete:368. COV.N^A15
Complete:369. SWY.N^A15
Complete:370. HCA.N


...

Complete:805. OK.N^D01
Complete:806. FI.N
Complete:807. SUB.N^C01
Complete:808. CTAS.OQ
Complete:809. FSR.N^B01
Complete:810. UK.N^B01
Complete:811. CGP.N^A01
Complete:812. TWX.N^A01
Complete:813. FLR_w.N^L00
Complete:814. AET_w.N^L00
Complete:815. SMI.N^I01
Complete:816. PRD.N^J01
Complete:817. NGH.N^L00
Complete:818. RML.N^H06
Complete:819. SYK.N
Complete:820. CCK.N
Complete:821. VO.N^L00
Complete:822. GRA.N^I21
Complete:823. BHMSQ.OB^A04
Complete:824. INTU.OQ
Complete:825. MET.N
Complete:826. RHI.N
Complete:827. AFS.N^L00
Complete:828. FPC.N^K00
Complete:829. FJ.N^K00
Complete:830. SEG.N^K00
Complete:831. PWJ.N^K00
Complete:832. CG.N^K00
Complete:833. EFU.N^K00
Complete:834. EOG.N
Complete:835. NI.N
Complete:836. UCM.N^J00
Complete:837. MKG.N^J00
Complete:838. YNR.N^J00
Complete:839. BFO.N^J00
Complete:840. MCO.N
Complete:841. OWC.N^L02
Complete:842. GAP.N^L10
Complete:843. DVN.N
Complete:844. NCE.N^H00
Complete:845. COMS.OQ^D10
Complete:846. RAD.N
Complete:847. UPR.N^G00
Complete:8

2023-09-04 13:59:50,205 P[7752] [MainThread 7748] HTTP TimeoutException: .


Complete:962. CHRS.OQ^F12
Complete:963. SK.N^E98
Complete:964. JH.N^E07
Complete:965. SEE.N
Complete:966. ITT.N^B98
Complete:967. PAS.N^C10
Complete:968. NTRS.OQ
Complete:969. CBB.N^A98
Complete:970. ECO.A^B03
Complete:971. BBI.N^A98
Complete:972. UEP.N^L97
Complete:973. AEE.N
Complete:974. FLM.N^D03
Complete:975. OMC.N
Complete:976. HFS.N^L97
Complete:977. CINF.OQ
Complete:978. BEV.N^L97
Complete:979. TFC.N
Complete:980. SB.N^K97
Complete:981. OEC.N^K97
Complete:982. FE.N
Complete:983. LLX.N^J97
Complete:984. SRR.N^H07
Complete:985. YUM.N
Complete:986. AMH.A^J97
Complete:987. KLAC.OQ
Complete:988. TDM.N^I97
Complete:989. NYN.N^H97
Complete:990. STT.N
Complete:991. NAE.N^H97
Complete:992. HBAN.OQ
Complete:993. ENS.N^H97


2023-09-04 14:03:04,172 P[7752] [MainThread 7748] Backend error. 400 Bad Request
2023-09-04 14:03:04,172 P[7752] [MainThread 7748] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:994. MD.N^H97
Complete:995. PGR.N
Complete:996. INGR.OQ^L06
Complete:997. SFS.N^H00
Complete:998. APA.OQ
Complete:999. GIDL.O^J97
Complete:1000. GWF.N^G97
Complete:1001. PEL.N^F97
Complete:1002. EFX.N
Complete:1003. USH.N^F97
Complete:1004. PVN.N^F97
Complete:1005. MS.N^F97
Complete:1006. SCHW.N
Complete:1007. CRR.N^F97
Complete:1008. CAH.N
Complete:1009. ADBE.OQ
Complete:1010. GLD.N^E97
Complete:1011. PAC.N^D97
Complete:1012. AAL.N^B97
Complete:1013. BOAT.O^D99
Complete:1014. LUB.N^E22
Complete:1015. RYAN.OQ^K06
Complete:1016. SHN.N^G00
Complete:1017. AZO.N
Complete:1018. TMO.N
Complete:1019. BLY.N^L96
Complete:1020. CNW.N^J15
Complete:1021. YELL.OQ^H23
Complete:1022. OM.N^I97
Complete:1023. VAT.N^I96
Complete:1024. VVI.N^G04
Complete:1025. TDY.N^H96
Complete:1026. OG.N^C01
Complete:1027. CAL.N
Complete:1028. USHC.O^G96
Complete:1029. AM^H13
Complete:1030. ROK.N
Complete:1031. RTN.N^L97
Complete:1032. SMS^G00
Complete:1033. CVS.N
Complete:1034. PMI.N^K99
Complete:1035. LOR.N^

In [16]:
# export the return excels
result.to_csv("return_all.csv")  

In [30]:
return_all = pd.read_csv("return_all.csv")

C:\Users\yeungwin\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
#df.to_csv("monthend_index.csv")

# Get Industry

In [17]:
industry_all, err =  ek.get_data(
    list(unique_constituents_list),
    fields = ['TR.TRBCBusinessSector','TR.TRBCEconomicSector' ]
    , parameters={'SDate':'1995-01-01','EDate':'2020-12-31'})

In [19]:
print(industry_all)

       Instrument           TRBC Business Sector Name  \
0             A.N     Healthcare Services & Equipment   
1     AABA.OQ^J19              Collective Investments   
2       AAL.N^B97                           Insurance   
3          AAL.OQ                      Transportation   
4           AAP.N                           Retailers   
...           ...                                 ...   
1147      ZBRA.OQ                Technology Equipment   
1148     ZE.N^E98                Technology Equipment   
1149      ZION.OQ       Banking & Investment Services   
1150    ZRN.N^F98          Cyclical Consumer Products   
1151        ZTS.N  Pharmaceuticals & Medical Research   

     TRBC Economic Sector Name  
0                   Healthcare  
1                   Financials  
2                   Financials  
3                  Industrials  
4           Consumer Cyclicals  
...                        ...  
1147                Technology  
1148                Technology  
1149              

In [20]:
# export the return excel
industry_all.to_csv("industry_all.csv") 

# 2. Data Return for DAX

In [6]:
ic, err = ek.get_data('.GDAXI', ['TR.IndexConstituentRIC'])

In [7]:
# get company list
ic, err = ek.get_data('.GDAXI', ['TR.IndexConstituentRIC'])
lj, err = ek.get_data('.GDAXI', 
                      ['TR.IndexJLConstituentChangeDate',
                       'TR.IndexJLConstituentRIC.change',
                       'TR.IndexJLConstituentRIC'],
                     {'SDate':'0D', 'EDate':'-31Y', 'IC':'B'}) ##start by 1957
lj['Date']=pd.to_datetime(lj['Date']).dt.date
lj.sort_values(['Date','Change'],ascending=False,inplace=True)
dates = [dt.date(1995,1,30)]
i = 0
while (dates[0] + dateutil.relativedelta.relativedelta(months=+i+1)) < dt.date.today():
    dates.append(dates[0] + dateutil.relativedelta.relativedelta(months=+i+1))
    i = i + 1
dates.append(dt.date.today())
df = pd.DataFrame(index=dates, columns=['Index Constituents'])

In [8]:
ic_list = ic['Constituent RIC'].tolist()
for i in range(len(dates)):
    print(str(dates[len(dates)-i-1]))
    df.at[dates[len(dates)-i-1],'Index Constituents'] = ic_list[:]
    for j in lj.index:
        if lj['Date'].loc[j] <= dates[len(dates)-i-1]:
            if lj['Date'].loc[j] > dates[len(dates)-i-2]:
                if lj['Change'].loc[j] == 'Joiner':
                    print('Removing ' + lj['Constituent RIC'].loc[j])
                    ic_list.remove(lj['Constituent RIC'].loc[j])
                elif lj['Change'].loc[j] == 'Leaver':
                    print('Adding ' + lj['Constituent RIC'].loc[j])
                    ic_list.append(lj['Constituent RIC'].loc[j])
            else:
                break

2023-12-01
2023-11-30
2023-10-30
2023-09-30
2023-08-30
2023-07-30
2023-06-30
2023-05-30
2023-04-30
2023-03-30
Adding FMEG.DE
Removing RHMG.DE
2023-02-28
Adding LIN.DE
Removing CBKG.DE
2023-01-30
2022-12-30
Adding PUMG.DE
Removing P911_p.DE
2022-11-30
2022-10-30
2022-09-30
Adding HFGG.DE
Removing ENR1n.DE
2022-08-30
2022-07-30
2022-06-30
Adding DHER.DE
Removing BEIG.DE
2022-05-30
2022-04-30
2022-03-30
Adding BEIG.DE
Adding ENR1n.DE
Removing DTGGe.DE
Removing HNRGn.DE
2022-02-28
2022-01-30
2021-12-30
Adding DTGGe.DE
Removing DTGGe.DE
2021-11-30
2021-10-30
Adding DWNG.DE
Removing BEIG.DE
2021-09-30
Removing PSHG_p.DE
Removing SHLG.DE
Removing BNRGn.DE
Removing SY1G.DE
Removing AIRG.DE
Removing PUMG.DE
Removing SATG_p.DE
Removing QIA.DE
Removing HFGG.DE
Removing ZALG.DE
Adding VTSCn.DE
Removing VTSCn.DE
2021-08-30
2021-07-30
2021-06-30
2021-05-30
2021-04-30
2021-03-30
Adding BEIG.DE
Removing ENR1n.DE
2021-02-28
2021-01-30
2020-12-30
2020-11-30
2020-10-30
2020-09-30
Adding ENR1n.DE
Removing

In [9]:
df.index = pd.to_datetime(df.index)

# Filter the DataFrame based on the date range to remove 2020 afterward data
start_date = datetime(1995, 1, 1)
end_date = datetime(2020, 12, 31)
filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]

unique_constituents = set()
for constituents_list in filtered_df["Index Constituents"]:
    unique_constituents.update(constituents_list)

# Convert the set back to a sorted list
unique_constituents_list = sorted(list(unique_constituents))
print(len(unique_constituents_list))

71


In [10]:
# download all return

i=0
result = pd.DataFrame()
for i in range(len(unique_constituents_list)): #company list from 19950101 to 20201231
    isSuccess = False
    while(not isSuccess):
        try:
            data_grid, err = ek.get_data(
            list(unique_constituents_list)[i],
            fields = ['TR.TotalReturn1D.date','TR.TotalReturn1D'],
            parameters={'SDate':'1995-01-01','EDate':'2020-12-31','Frq':'D','CH':'Fd'}
            )
            isSuccess = True
        except ek.EikonError as err:
            continue
        result = pd.concat([result,data_grid],axis=0).reset_index(drop=True)
        print('Complete:'+ str(i) + '. '+list(pd.unique(lj['Constituent RIC']))[i])
        i+=1

Complete:0. FMEG.DE
Complete:1. RHMG.DE
Complete:2. LIN.DE
Complete:3. CBKG.DE
Complete:4. PUMG.DE
Complete:5. P911_p.DE
Complete:6. HFGG.DE
Complete:7. ENR1n.DE
Complete:8. DHER.DE
Complete:9. BEIG.DE
Complete:10. DTGGe.DE
Complete:11. HNRGn.DE
Complete:12. DWNG.DE
Complete:13. PSHG_p.DE
Complete:14. SHLG.DE
Complete:15. BNRGn.DE
Complete:16. SY1G.DE
Complete:17. AIRG.DE
Complete:18. SATG_p.DE
Complete:19. QIA.DE
Complete:20. ZALG.DE
Complete:21. VTSCn.DE
Complete:22. WDIG.DE^A21
Complete:23. LHAG.DE
Complete:24. TKAG.DE
Complete:25. MTXGn.DE
Complete:26. LIN1.DE^J18
Complete:27. PSMGn.DE
Complete:28. 1COV.DE
Complete:29. LING.DE^D19
Complete:30. DB11.DE^D17
Complete:31. DB1Gn.DE
Complete:32. UN01.DE
Complete:33. SDFGn.DE
Complete:34. LXSG.DE
Complete:35. VNAn.DE
Complete:36. OSRn.DE^G21
Complete:37. MANG.DE^I21
Complete:38. CECG.DE
Complete:39. CONG.DE
Complete:40. DB1Gne.DE^B12


2023-12-01 10:15:33,401 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:15:33,401 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:41. FREG.DE
Complete:42. FREG_p.DE^A11
Complete:43. SZGG.DE
Complete:44. HEIG.DE
Complete:45. VOWG.DE
Complete:46. VOWG_p.DE
Complete:47. BAYGn.DE
Complete:48. IFXGn.DE
Complete:49. DPBGn.DE^L15
Complete:50. HRXG.DE^J09
Complete:51. TUI1n.DE
Complete:52. ALTG.DE^H10
Complete:53. MRCG.DE
Complete:54. SCHG.DE^I08
Complete:55. HVMG.DE^I08
Complete:56. MLPG.DE


2023-12-01 10:17:01,039 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:17:01,040 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:57. EPCGn.DE^J09
Complete:58. DGXG.DE^I06
Complete:59. MLPG_p.DE^I01
Complete:60. DRSDn.DE^G02
Complete:61. SAPG_p.DE^F01
Complete:62. SAPG.DE
Complete:63. AROG.DE^C18


2023-12-01 10:17:35,998 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:17:35,999 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:64. DHLn.DE
Complete:65. VIAG.DE^F00
Complete:66. VDFGn.DE^H02
Complete:67. HOEG.DE^G05
Complete:68. DAIGa.DE^K98
Complete:69. MBGn.DE
Complete:70. DAIG.DE^L98


In [11]:
df.head()

,Index Constituents
1995-01-30,"[BASFn.DE, SIEGn.DE, EONGn.DE, DBKGn.DE, RWEG...."
1995-02-28,"[BASFn.DE, SIEGn.DE, EONGn.DE, DBKGn.DE, RWEG...."
1995-03-30,"[BASFn.DE, SIEGn.DE, EONGn.DE, DBKGn.DE, RWEG...."
1995-04-30,"[BASFn.DE, SIEGn.DE, EONGn.DE, DBKGn.DE, RWEG...."
1995-05-30,"[BASFn.DE, SIEGn.DE, EONGn.DE, DBKGn.DE, RWEG...."


In [12]:
result.loc[result['Date'].notnull()]

,Instrument,Date,Daily Total Return
0,1COV.DE,,<NA>
1,1COV.DE,,<NA>
2,1COV.DE,,<NA>
3,1COV.DE,,<NA>
4,1COV.DE,,<NA>
...,...,...,...
468716,WDIG.DE^A21,2020-12-22T00:00:00Z,-1.340415
468717,WDIG.DE^A21,2020-12-23T00:00:00Z,-2.076391
468718,WDIG.DE^A21,2020-12-28T00:00:00Z,-13.612565
468719,WDIG.DE^A21,2020-12-29T00:00:00Z,-2.212121


In [13]:
# export the return excels
result.to_csv("dax_return_all.csv")  

In [14]:
industry_all, err =  ek.get_data(
    list(unique_constituents_list),
    fields = ['TR.TRBCBusinessSector','TR.TRBCEconomicSector' ]
    , parameters={'SDate':'1995-01-01','EDate':'2020-12-31'})

In [15]:
print(industry_all)

     Instrument         TRBC Business Sector Name TRBC Economic Sector Name
0       1COV.DE                         Chemicals           Basic Materials
1      ADSGn.DE        Cyclical Consumer Products        Consumer Cyclicals
2   ALTG.DE^H10                         Chemicals           Basic Materials
3       ALVG.DE                         Insurance                Financials
4   AROG.DE^C18                         Retailers        Consumer Cyclicals
..          ...                               ...                       ...
66  VIAG.DE^F00                         Utilities                 Utilities
67      VNAn.DE                       Real Estate               Real Estate
68      VOWG.DE          Automobiles & Auto Parts        Consumer Cyclicals
69    VOWG_p.DE          Automobiles & Auto Parts        Consumer Cyclicals
70  WDIG.DE^A21  Industrial & Commercial Services               Industrials

[71 rows x 3 columns]


In [16]:
# export the return excel
industry_all.to_csv("dax_industry_all.csv") 

In [17]:
List_Length = df['Index Constituents'].apply(len)
List_Length

1995-01-30    32
1995-02-28    32
1995-03-30    32
1995-04-30    32
1995-05-30    32
              ..
2023-08-30    40
2023-09-30    40
2023-10-30    40
2023-11-30    40
2023-12-01    40
Name: Index Constituents, Length: 348, dtype: int64

In [18]:
filtered_date = []
start_date = datetime(1995, 1, 1).date()
end_date = datetime(2020, 12, 31).date()
for date_obj in dates:
    if start_date <= date_obj <= end_date:
        filtered_date.append(date_obj)
date_strings = [date.strftime('%Y%m%d') for date in filtered_date]
#date_strings

In [19]:
# download company in DAX for each end of month
i=0
result_df = pd.DataFrame()
for date_str in date_strings:
    isSuccess = False
    while(not isSuccess):
        try:
            df, e = ek.get_data(f'0#.GDAXI({date_str})', ['TR.RIC'])
            df['Date']=date_str
            isSuccess = True
        except ek.EikonError as err:
            continue
        df = df.drop(['Instrument'],axis=1)
        result_df = pd.concat([result_df, df], ignore_index=True)
        print('Complete:'+ str(date_str))
print(result_df)

Complete:19950130
Complete:19950228
Complete:19950330
Complete:19950430
Complete:19950530
Complete:19950630
Complete:19950730
Complete:19950830
Complete:19950930
Complete:19951030
Complete:19951130
Complete:19951230
Complete:19960130
Complete:19960229
Complete:19960330
Complete:19960430
Complete:19960530
Complete:19960630
Complete:19960730
Complete:19960830
Complete:19960930
Complete:19961030
Complete:19961130
Complete:19961230
Complete:19970130
Complete:19970228
Complete:19970330
Complete:19970430
Complete:19970530
Complete:19970630
Complete:19970730
Complete:19970830
Complete:19970930
Complete:19971030
Complete:19971130
Complete:19971230
Complete:19980130
Complete:19980228
Complete:19980330
Complete:19980430
Complete:19980530
Complete:19980630
Complete:19980730
Complete:19980830
Complete:19980930
Complete:19981030
Complete:19981130
Complete:19981230
Complete:19990130
Complete:19990228
Complete:19990330
Complete:19990430
Complete:19990530
Complete:19990630
Complete:19990730
Complete:1

2023-12-01 10:22:36,053 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:22:36,054 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:20110830


2023-12-01 10:22:55,249 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:22:55,250 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request
2023-12-01 10:23:13,402 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:23:13,403 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request
2023-12-01 10:23:31,546 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:23:31,547 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:20110930
Complete:20111030
Complete:20111130
Complete:20111230
Complete:20120130
Complete:20120229
Complete:20120330
Complete:20120430
Complete:20120530
Complete:20120630
Complete:20120730
Complete:20120830
Complete:20120930
Complete:20121030
Complete:20121130
Complete:20121230
Complete:20130130
Complete:20130228
Complete:20130330
Complete:20130430
Complete:20130530
Complete:20130630
Complete:20130730
Complete:20130830
Complete:20130930


...

Complete:20131130
Complete:20131230
Complete:20140130
Complete:20140228
Complete:20140330
Complete:20140430
Complete:20140530
Complete:20140630
Complete:20140730
Complete:20140830
Complete:20140930
Complete:20141030
Complete:20141130
Complete:20141230
Complete:20150130
Complete:20150228
Complete:20150330
Complete:20150430
Complete:20150530
Complete:20150630
Complete:20150730


2023-12-01 10:27:20,503 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:27:20,504 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request
2023-12-01 10:27:38,649 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:27:38,650 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:20150830
Complete:20150930
Complete:20151030


2023-12-01 10:27:59,819 P[5956] [MainThread 2128] Backend error. 400 Bad Request
2023-12-01 10:27:59,820 P[5956] [MainThread 2128] HTTP request failed: EikonError-Backend error. 400 Bad Request


Complete:20151130
Complete:20151230
Complete:20160130
Complete:20160229
Complete:20160330
Complete:20160430
Complete:20160530
Complete:20160630
Complete:20160730
Complete:20160830
Complete:20160930
Complete:20161030
Complete:20161130
Complete:20161230
Complete:20170130
Complete:20170228
Complete:20170330
Complete:20170430
Complete:20170530
Complete:20170630
Complete:20170730
Complete:20170830
Complete:20170930
Complete:20171030
Complete:20171130
Complete:20171230
Complete:20180130
Complete:20180228
Complete:20180330
Complete:20180430
Complete:20180530
Complete:20180630
Complete:20180730
Complete:20180830
Complete:20180930
Complete:20181030
Complete:20181130
Complete:20181230
Complete:20190130
Complete:20190228
Complete:20190330
Complete:20190430
Complete:20190530
Complete:20190630
Complete:20190730
Complete:20190830
Complete:20190930
Complete:20191030
Complete:20191130
Complete:20191230
Complete:20200130
Complete:20200229
Complete:20200330
Complete:20200430
Complete:20200530
Complete:2

In [20]:
aa = result_df.groupby('Date')['RIC'].nunique().reset_index()
aa

,Date,RIC
0,19950130,31
1,19950228,31
2,19950330,31
3,19950430,31
4,19950530,31
...,...,...
307,20200830,30
308,20200930,30
309,20201030,30
310,20201130,30


In [21]:
result_df.to_csv('dax_month_end.csv')